# Local Outlier Factor sur clusters
---
_Tentative d'affinement de cluster réalisé avec des applications de Kmean et SOM sur un dataset avec LOF_

---

  1. Procédure automatisé
  2. Exploration
---

### 1. Procédure automatisée
- importation des libraries nécessaires numpy, gensim, sklearn, ...

In [1]:
import random
import numpy as np
from gensim.models import Word2Vec, KeyedVectors
from sklearn.neighbors import LocalOutlierFactor
from sklearn.decomposition import PCA
from os import listdir, path, makedirs
import re

- fonction d'évaluation pour évaluer rapidement chaque cluster:

In [2]:

def loadDataset(file, dico):
    """Charge un fichier keywords et le transforme en dictionnaire

    Args:
        file (fichier): fichier keywords correspondant
        dico (dictionnaire): dictionnaire vide
    """
    with open(file, 'r') as file:
        for line in file:
            resultat[line.split("\n")[0]] = 0

def recordic(file, dico):
    """Parcours un dataset et vérifie le nombre d'occurence des clefs d'un dictionnaire 

    Args:
        file (fichier): dataset
        dico (dictionnaire): dictionnaire de mot clés
    """
    dico["LINE"] = 0
    dico["OCCUR"] = 0
    with open(file, 'r') as file:
        for e in file:
            dico["LINE"] += 1
            for k in dico.keys():
                if len(k.split(" ")) == 1:
                    for word in e.split(" "):
                        word = re.sub('[^a-z0-9àâäéèëêïîôöùûüÿç]+', '', word.lower())
                        if k == word:
                            dico[k] +=1
                            dico["OCCUR"]  +=1
                else:
                    if(k in e):
                        dico[k] +=1
                        dico["OCCUR"] +=1

def rapportR(dico, out):
    """représente le rapport sur un dataset en fonction des mots clés relatif ou constitutif de sa formation

    Args:
        dico (dictionnaire): dictionnaire des mots clés
        out (fichier): fichier de sortie
    """
    with open(out, 'a') as file:
        for k in dico.keys():
            if(k != "LINE") and (dico['LINE']>0):
                file.write(f"{k}:{dico[k]}/{round(dico[k]/dico['LINE']*100, 4)}%\n")
        if dico['LINE']>0:
            file.write(f"Total occurences:{dico['OCCUR']}/{round(dico['OCCUR']/dico['LINE']*100, 4)}%\n")


- fonction d'encodage

In [3]:
def encode(msg, model, dim):
    """
        Encode un message:
    Args:
        msg (string): une chaîne de caractère correspondant aux données à encodées
        model (array): modéle d'encodage
        dim: nombre de dimensions nécessaires

    Returns:
        array d'int: vecteur de données encodées
    
    """
    return np.mean([model[word] for word in msg if word in model] or [np.zeros(dim)], axis = 0)

- fonction de sauvegarde

In [4]:
#création de dossier
def SaveLofSelect(file, index, datas):
    with open(file, 'a') as out:
        for i,e in enumerate(index):
            if e<0:
                out.write(datas[i])

- Ensemble d'élément pour le parcours:

In [5]:
paths = ["1M_Lem_clusters/", "1M_Min_clusters/", "1M_MinStp_clusters/", "1M_Std_clusters/"]
version = ["km_25/", "km_50/", "km_100/", "som_100/", "som_225/", "som_400/"]
lof = ["lofk2", "lofk3", "lofk4"]

In [6]:
for i in range(len(paths)):
    for j in range(len(version)):
        CurrentPath = paths[i]+version[j]
        clean = []
        raw = []
        #récupération de la liste des fichier clean & raw:
        for e in listdir(CurrentPath):
            if e.endswith("clean_msg.txt"):
                clean.append(e)
            elif e.endswith("raw_msg.txt"):
                raw.append(e)
            else:
                pass
        #traitement pour chaque cluster:
        for k in range(len(clean)):
            corpus = CurrentPath+clean[k]
            corpus_raw = CurrentPath+raw[k]
            model = Word2Vec(corpus_file=corpus, vector_size=300, window=5, min_count=2, workers=8, sg=1)
            wv = model.wv
            wv.save("w2vec_model_temp")
            model = KeyedVectors.load("w2vec_model_temp")
            #chargement des données en mémoire:
            FormatDataset = []
            RawData = []
            with open(corpus, "r") as file:
                lines = file.readlines()
    
            FormatDataset = [line.split() for line in lines]

            with open(corpus_raw, "r") as file:
                lines = file.readlines()
    
            RawData = [line for line in lines]
            #encore les données:
            dataset_encoded = np.array([encode(msg, model, 300) for msg in FormatDataset])
            #reduction de dimensions:
            pca = PCA(n_components=2)
            model2d = pca.fit_transform(dataset_encoded)
            #application en fonction du kNeighbots choisi
            for l in range(len(lof)):
                clf = LocalOutlierFactor(n_neighbors= l+2)
                out = clf.fit_predict(model2d)
                path_test = CurrentPath+lof[l]+"/" 
                if not path.isdir(path_test):
                    makedirs(path_test)
                outlier = CurrentPath+lof[l]+"/"+clean[k][:-4]+"_"+lof[l]+".txt"
                SaveLofSelect(outlier, out, RawData)
                resultat = {}
                loadDataset("keywords",resultat)
                recordic(outlier ,resultat)
                rapportR(resultat, outlier[:-4]+"_rapport.txt")

---

### 2. Exploration

- Meilleurs résultats par dataset et procédures:

1M_Lem_clusters

    Algorithme:
        LOF                      Nbr                 %              infos

    km25:
        lof kNeighbors = 2:
        cluster 17               115                 2.2896%         Expressions prépondérantes: salope, pute, pétasse
        lof kNeighbors = 3:
        cluster 17                90                 2.5989%         Expressions prépondérantes: salope, pute, pétasse
        cluster 24                 9                 1.0169%         Expressions prépondérantes: *
        lof kNeighbors = 4:
        cluster 17                92                 3.2987%         Expressions prépondérantes: salope, pute, pétasse
        cluster 22                18                 0.6218%         Expressions prépondérantes: *
       
    km50:
        lof kNeighbors = 2:
        cluster 17               113                 2.2546%         Expressions prépondérantes: salope, pute
        cluster 19                50                 2.1570%         Expressions prépondérantes: pétasse
        cluster 42                 7                 2.9536%         Expressions prépondérantes: *
        lof kNeighbors = 3:
        cluster 17                85                 3.0390%         Expressions prépondérantes: salope, pute
        cluster 19                52                 3.0642%         Expressions prépondérantes: pétasse
        cluster 42                 7                 3.7433%         Expressions prépondérantes: *
        lof kNeighbors = 4:
        cluster 17                74                 3.8422%         Expressions prépondérantes: salope, pute
        cluster 19                51                 3.5003%         Expressions prépondérantes: pétasse
        cluster 42                 6                 4.2533%         Expressions prépondérantes: *
        
    km100:
        lof kNeighbors = 2:
        cluster  1                 3                 2.5862%         Expressions prépondérantes: *
        cluster 30                 5                 1.4535%         Expressions prépondérantes: *
        cluster 38                 2                 1.2903%         Expressions prépondérantes: *
        cluster 45                17                 5.3797%         Expressions prépondérantes: pute
        cluster 54                72                 7.0796%         Expressions prépondérantes: pute, salope
        cluster 75                42                 3.7975%         Expressions prépondérantes: pétasse
        cluster 97                 6                 2.0339%         Expressions prépondérantes: *
        lof kNeighbors = 3: 
        cluster  1                 2                 1.626%          Expressions prépondérantes: *
        cluster 30                 4                 1.8349%         Expressions prépondérantes: *
        cluster 45                 8                 3.6199%         Expressions prépondérantes: *
        cluster 54                49                 6.3144%         Expressions prépondérantes: pute
        cluster 75                38                 4.0084%         Expressions prépondérantes: pétasse
        cluster 97                 4                 1.8433%         Expressions prépondérantes: *
        lof kNeighbors = 4:
        cluster 30                 4                 2.3256%         Expressions prépondérantes: *
        cluster 45                 8                 5.0%            Expressions prépondérantes: *
        cluster 54                47                 6.4828%         Expressions prépondérantes: pute
        cluster 75                41                 4.6328%         Expressions prépondérantes: pétasse
        cluster 97                 4                 2.0408%         Expressions prépondérantes: *
        
    som100:
        lof kNeighbors = 2:
        cluster  0                 4                 1.2121%         Expressions prépondérantes: *
        cluster 34                 3                 2.2901%         Expressions prépondérantes: *
        cluster 36                 3                 1.1321%         Expressions prépondérantes: *
        cluster 41                 4                 1.278%          Expressions prépondérantes: *
        cluster 54                36                 5.6426%         Expressions prépondérantes: pétasse
        cluster 84                 5                 1.9231%         Expressions prépondérantes: *
        cluster 90                28                 6.9479%         Expressions prépondérantes: pute
        cluster 94                 4                 1.6327%         Expressions prépondérantes: *
        cluster 96                 2                 1.7241%         Expressions prépondérantes: *
        lof kNeighbors = 3:
        cluster 34                 2                 1.3986%         Expressions prépondérantes: *
        cluster 36                 2                 1.1628%         Expressions prépondérantes: *
        cluster 41                 3                 1.5385%         Expressions prépondérantes: *
        cluster 54                37                 6.8646%         Expressions prépondérantes: pétasse
        cluster 60                24                 1.3675%         Expressions prépondérantes: pute
        cluster 84                 2                 1.0152%         Expressions prépondérantes: *
        cluster 94                 4                 2.5974%         Expressions prépondérantes: *
        lof kNeighbors = 4:
        cluster  0                 2                 1.1299%         Expressions prépondérantes: *
        cluster 36                 2                 1.4815%         Expressions prépondérantes: *
        cluster 42                 2                 1.3333%         Expressions prépondérantes: *
        cluster 54                37                 6.789%          Expressions prépondérantes: pétasse
        cluster 60                25                 1.5509%         Expressions prépondérantes: pute
        cluster 84                 2                 1.1111%         Expressions prépondérantes: *
        cluster 88                 2                 1.3245%         Expressions prépondérantes: *
        cluster 90                23                 8.3942%         Expressions prépondérantes: pute
        cluster 94                 4                 3.3058%         Expressions prépondérantes: *
        cluster 97                 2                 1.2987%         Expressions prépondérantes: *
        
    som225:
        lof kNeighbors = 2:
        cluster 17                 1                 1.2048%         Expressions prépondérantes: *
        cluster 35                17                 4.8711%         Expressions prépondérantes: pute
        cluster 65                 1                 1.6129%         Expressions prépondérantes: *
        cluster 66                 1                 1.2821%         Expressions prépondérantes: *
        cluster 99                52                14.2077%         Expressions prépondérantes: pute, salope
        cluster 107               39                 3.2473%         Expressions prépondérantes: pétasse
        cluster 133                3                 2.2222%         Expressions prépondérantes: *
        cluster 167                1                 1.4085%         Expressions prépondérantes: *
        cluster 205                1                 1.5385%         Expressions prépondérantes: *
        cluster 215                1                 1.3889%         Expressions prépondérantes: *
        lof kNeighbors = 3:
        cluster  3                 1                 1.5152%         Expressions prépondérantes: *
        cluster 35                15                 6.0976%         Expressions prépondérantes: pute
        cluster 62                 4                 3.125%          Expressions prépondérantes: *
        cluster 65                 1                 1.5625%         Expressions prépondérantes: *
        cluster 99                57                16.2393%         Expressions prépondérantes: pute, enculé
        cluster 107               37                 3.8382%         Expressions prépondérantes: pétasse
        cluster 133                2                 1.5152%         Expressions prépondérantes: *
        cluster 158                3                 2.7523%         Expressions prépondérantes: *
        cluster 172                1                 1.5625%         Expressions prépondérantes: *
        cluster 205                1                 1.9231%         Expressions prépondérantes: *
        lof kNeighbors = 4:
        cluster 17                 1                 1.9231%         Expressions prépondérantes: *
        cluster 35                 7                 3.6082%         Expressions prépondérantes: *
        cluster 62                 4                 2.6846%         Expressions prépondérantes: *
        cluster 65                 2                 2.7778%         Expressions prépondérantes: *
        cluster 84                 6                 3.1746%         Expressions prépondérantes: *
        cluster 99                57                17.4847%         Expressions prépondérantes: pute
        cluster 107               37                 4.0839%         Expressions prépondérantes: pétasse
        cluster 132                2                 1.6667%         Expressions prépondérantes: *
        cluster 146                5                 1.6556%         Expressions prépondérantes: *
        cluster 158                3                 3.4091%         Expressions prépondérantes: *

    som400:
        lof kNeighbors = 2:
        cluster 132               10                 6.993%          Expressions prépondérantes: pute
        cluster 167               51                30.1775%         Expressions prépondérantes: pute
        cluster 176                2                 3.4483%         Expressions prépondérantes: *
        cluster 193                8                 4.1237%         Expressions prépondérantes: *
        cluster 209               14                 4.5902%         Expressions prépondérantes: pute
        cluster 245               36                12.1212%         Expressions prépondérantes: pétasse
        cluster 277                2                 2.2727%         Expressions prépondérantes: *
        cluster 303                2                 2.0833%         Expressions prépondérantes: *
        cluster 310                1                 4.3478%         Expressions prépondérantes: *
        cluster 318                1                 2.9412%         Expressions prépondérantes: *
        lof kNeighbors = 3:
        cluster 123                1                 1.9231%         Expressions prépondérantes: *
        cluster 132               10                 6.8966%         Expressions prépondérantes: pute
        cluster 167               57                33.5294%         Expressions prépondérantes: pute
        cluster 176                2                 3.2787%         Expressions prépondérantes: *
        cluster 193                4                 2.2222%         Expressions prépondérantes: *
        cluster 209               13                 5.9361%         Expressions prépondérantes: pétasse
        cluster 263                1                 2.0%            Expressions prépondérantes: *
        cluster 282                3                 2.8302%         Expressions prépondérantes: *
        cluster 310                2                 7.1429%         Expressions prépondérantes: *
        cluster 318                1                 3.3333%         Expressions prépondérantes: *
        lof kNeighbors = 4:
        cluster 123                1                 2.3256%         Expressions prépondérantes: *
        cluster 132                8                 5.1282%         Expressions prépondérantes: *
        cluster 156                2                 2.6316%         Expressions prépondérantes: *
        cluster 167               58                36.7089%         Expressions prépondérantes: pute
        cluster 193                4                 2.2989%         Expressions prépondérantes: *
        cluster 209               11                 6.0109%         Expressions prépondérantes: salope
        cluster 282                3                 3.7037%         Expressions prépondérantes: *
        cluster 297                1                 2.381%          Expressions prépondérantes: *
        cluster 310                1                 4.5455%         Expressions prépondérantes: *
        cluster 311                2                 2.4096%         Expressions prépondérantes: *

cluster exemple: som400 k4 84:

xptddddddddr avec plaisir mon chat

Mon plaisir🖤 https://t.co/al545hiuAx

#Battlefield2042XMicromania je participe avec plaisir

Oui avec plaisir 😁

recevoir une commande quel plaisir

omg avec plaisir 🧎‍♀️

Merci 🙏🏾 ça fait plaisir

#TEAMG1 ca ferait plaisir a noel !

Avec plaisir mon reuf

Avec c plaisir 😊

Avec plaisir ma Camille ❤️❤️😘😘

Avec plaisir ma guerrière ☺️ à la tienne 🌹🥂🍸🤗😘

Au plaisir de vous y voir! https://t.co/LEURMyI3mK

Arf merci ça me fait plaisir ça

Ça me fait plaisir 🤣

Ca fait plaisir merci 😌

Je participe avec plaisir #Battlefield2042XMicromania 🤞

Avec plaisir Dom 😘😘✨✨💫

Avec plaisir chou ❤️

C’était un plaisir les reufs

Avec plaisir mon chat

#TEAMG1 ca fera plaisir a noel !

omg oui quel plaisir

Je participe avec plaisir 🤞❄️🎅 #ResidentEvilLeFilm

Merci beaucoup chère Dominique 🌺Avec plaisir 😊🤗Bisous 😘

Ça aurait été avec un immense plaisir 🐓

Je participe avec plaisir au concours.

Bonjour, merci, je tente ma chance avec plaisir 🙂

Bonjour je participe avec grand plaisir

Avec plaisir bisous 😘😍❤️🤗

---

_Conclusion des cluster sur un dataset lemmatisé:
    On Constate que même si on arrive à avoir des clusters avec une représentation plus élevée de vocabulaire pouvant constituer des messages haineux le nombre reste moindre. En parcourant les clusters, on remarque à défaut d'avoir une démarcation claire sur les éléments recherchés, qu'il y a bien souvent des "thématiques" qui s'articulent autour d'une suite de caractère spécifique, un verbe ou une expression._
    
---

1M_Std_clusters:

    km25:
        lof kNeighbors = 2:
        cluster  1                 65            1.3919%          Expressions prépondérantes: pute, salope ,beurette
        cluster 21                 93            1.3581%          Expressions prépondérantes: pétasse, pute
        lof kNeighbors = 3:
        cluster  1                 32            1.0771%          Expressions prépondérantes: pute
        cluster 21                 66            1.7713%          Expressions prépondérantes: pétasse, pute
        lof kNeighbors = 4:
        cluster 21                 63            2.5271%          Expressions prépondérantes: pétasse, pute
        
    km50:
        lof kNeighbors = 2:                 
        cluster 32                  4            2.5478%          Expressions prépondérantes: *
        cluster 33                 37           13.7037%          Expressions prépondérantes: pétasse
        cluster 34                121            2.8238%          Expressions prépondérantes: pute
        lof kNeighbors = 3:
        cluster 32                  4            2.5641%          Expressions prépondérantes: *
        cluster 33                 36           17.1429%          Expressions prépondérantes: pétasse
        cluster 34                 79            3.2807%          Expressions prépondérantes: pute
        lof kNeighbors = 4:
        cluster  7                  4            1.2232%          Expressions prépondérantes: *
        cluster 32                  4            3.2%             Expressions prépondérantes: *
        cluster 33                 36           20.4545%          Expressions prépondérantes: pétasse
        cluster 34                 66            3.6145%          Expressions prépondérantes: pute
        
    km100:
        lof kNeighbors = 2:
        cluster 29                  4            1.8779%          Expressions prépondérantes: *
        cluster 66                  5            3.7594%          Expressions prépondérantes: *
        cluster 69                  5            1.1574%          Expressions prépondérantes: *
        cluster 83                 35            5.117%           Expressions prépondérantes: pétasse
        cluster 90                 62            8.5048%          Expressions prépondérantes: pute
        lof kNeighbors = 3:
        cluster 29                  4            2.2989%          Expressions prépondérantes: *
        cluster 66                  4            3.3333%          Expressions prépondérantes: *
        cluster 83                 35            6.3636%          Expressions prépondérantes: pétasse
        cluster 90                 48            8.4063%          Expressions prépondérantes: pute
        cluster 92                  4            1.0471%          Expressions prépondérantes: *
        cluster 97                  2            1.3605%          Expressions prépondérantes: *
        cluster 99                  2            2.5974%          Expressions prépondérantes: *
        lof kNeighbors = 4:
        cluster 33                  3            1.4019%          Expressions prépondérantes: *
        cluster 66                  4            3.7736%          Expressions prépondérantes: *
        cluster 69                  3            1.5707%          Expressions prépondérantes: *
        cluster 83                 36            7.0039%          Expressions prépondérantes: pétasse
        cluster 90                 47            8.8346%          Expressions prépondérantes: pute
        cluster 92                  5            1.6287%          Expressions prépondérantes: *
        cluster 99                  2            3.125%           Expressions prépondérantes: *
        
    som100:
        lof kNeighbors = 2:   
        cluster 12                  4            1.3514%          Expressions prépondérantes: *
        cluster 22                 35           11.1821%          Expressions prépondérantes: pétasse
        cluster 24                  7            1.4523%          Expressions prépondérantes: *
        cluster 27                 35            1.8909%          Expressions prépondérantes: pute, nègre
        cluster 47                  2            1.087%           Expressions prépondérantes: *
        cluster 53                  5            2.5126%          Expressions prépondérantes: *
        cluster 55                 31            3.0723%          Expressions prépondérantes: pute
        cluster 78                  6            2.5532%          Expressions prépondérantes: *
        lof kNeighbors = 3:
        cluster 22                 36           17.561%           Expressions prépondérantes: pétasse
        cluster 24                  6            2.0619%          Expressions prépondérantes: *
        cluster 27                 20            1.8501%          Expressions prépondérantes: pute, nègre
        cluster 47                  2            1.5625%          Expressions prépondérantes: *
        cluster 53                  8            4.7337%          Expressions prépondérantes: *
        cluster 55                 32            4.8338%          Expressions prépondérantes: pute
        cluster 78                  3            1.5789%          Expressions prépondérantes: *
        lof kNeighbors = 4:
        cluster 22                 36           21.6867%          Expressions prépondérantes: pétasse
        cluster 24                  5            2.451%           Expressions prépondérantes: *
        cluster 27                 18            2.2305%          Expressions prépondérantes: pute, nègre
        cluster 53                  6            3.4483%          Expressions prépondérantes: *
        cluster 55                 22            3.9711%          Expressions prépondérantes: pute
        cluster 72                  1            1.0638%          Expressions prépondérantes: *
        cluster 78                  3            1.9108%          Expressions prépondérantes: *
        cluster 91                  1            1.0204%          Expressions prépondérantes: *
        
    som225:
        lof kNeighbors = 2:  
        cluster  17                 5            1.938%           Expressions prépondérantes: *
        cluster  59                 1            1.6129%          Expressions prépondérantes: *
        cluster  81                 2            1.4388%          Expressions prépondérantes: *
        cluster  89                 3            4.4118%          Expressions prépondérantes: *
        cluster  98                36           15.6522%          Expressions prépondérantes: pétasse
        cluster 105                48           33.8028%          Expressions prépondérantes: pute
        cluster 117                36            1.9038%          Expressions prépondérantes: pute, salope
        cluster 121                 2            1.9417%          Expressions prépondérantes: *
        cluster 150                 2            2.9412%          Expressions prépondérantes: *
        cluster 162                 2            2.7027%          Expressions prépondérantes: *
        lof kNeighbors = 3:
        cluster  17                 5            2.8409%          Expressions prépondérantes: *
        cluster  59                 1            1.6949%          Expressions prépondérantes: *
        cluster  81                 3            2.4194%          Expressions prépondérantes: *
        cluster  89                 1            1.2346%          Expressions prépondérantes: *
        cluster  98                37           22.2892%          Expressions prépondérantes: pétasse
        cluster 105                42           30.2158%          Expressions prépondérantes: pute
        cluster 117                19            1.732%           Expressions prépondérantes: salope, bitch
        cluster 121                 2            2.1277%          Expressions prépondérantes: *
        cluster 150                 3            5.0%             Expressions prépondérantes: *
        cluster 162                 2            3.0303%          Expressions prépondérantes: *
        lof kNeighbors = 4:
        cluster  17                 4            3.252%           Expressions prépondérantes: *
        cluster  59                 1            2.2222%          Expressions prépondérantes: *
        cluster  81                 3            2.521%           Expressions prépondérantes: *
        cluster  89                 1            1.3333%          Expressions prépondérantes: *
        cluster  98                37            24.8322%         Expressions prépondérantes: pétasse
        cluster 105                27            23.8938%         Expressions prépondérantes: pute
        cluster 117                13            1.5625%          Expressions prépondérantes: pute, bitch
        cluster 121                 3            3.7975%          Expressions prépondérantes: *
        cluster 150                 3            5.1724%          Expressions prépondérantes: *
        cluster 162                 1            1.6393%          Expressions prépondérantes: *
        
    som400:
        lof kNeighbors = 2:
        cluster  47                 2            3.5088%          Expressions prépondérantes: *
        cluster  94                 1            7.1429%          Expressions prépondérantes: *
        cluster 105                27            23.8938%         Expressions prépondérantes: pute
        cluster 266                 2            2.5%             Expressions prépondérantes: *
        cluster 289                23           40.3509%          Expressions prépondérantes: pute
        cluster 291                 2            6.8966%          Expressions prépondérantes: *
        cluster 332                18           15.7895%          Expressions prépondérantes: pute, salope
        cluster 349                41           33.3333%          Expressions prépondérantes: pute, salope
        cluster 385                35           16.055%           Expressions prépondérantes: pétasse
        cluster 389                 2            5.4054%          Expressions prépondérantes: *
        lof kNeighbors = 3:
        cluster  47                 2            3.4483%          Expressions prépondérantes: *
        cluster  94                 1           11.1111%          Expressions prépondérantes: *
        cluster 266                 2            2.9412%          Expressions prépondérantes: *
        cluster 289                23           39.6552%          Expressions prépondérantes: pute
        cluster 291                 1            3.2258%          Expressions prépondérantes: *
        cluster 332                12           15.5844%          Expressions prépondérantes: pute, salope
        cluster 349                42           36.8421%          Expressions prépondérantes: pute, salope
        cluster 385                36           20.6897%          Expressions prépondérantes: pétasse
        cluster 389                 1            3.5714%          Expressions prépondérantes: *
        lof kNeighbors = 4:
        cluster  47                 2            3.7736%          Expressions prépondérantes: *
        cluster  94                 1           10.0%             Expressions prépondérantes: *
        cluster 266                 1            1.5152%          Expressions prépondérantes: *
        cluster 289                24           45.283%           Expressions prépondérantes: pute
        cluster 291                 2            6.4516%          Expressions prépondérantes: *
        cluster 332                 8           11.4286%          Expressions prépondérantes: *
        cluster 349                40           40.404%           Expressions prépondérantes: pute, salope
        cluster 385                36           22.6415%          Expressions prépondérantes: pétasse
        cluster 389                 1            2.5%             Expressions prépondérantes: *
           

contenu du cluster som400 lof k 4 289:

Mon fils avec mark

Fils de pute https://t.co/GbThVyfWfZ https://t.co/2FjJfPq9fO

et ça va chui un fils de pute si j’le regarde pas

Oui mais pas de fils de connard comme toi !

Cringe tu est un fils de merde

Merci de remplir mon fil d'amour !

Fila jamais te critiquei

mon fils il fera du foot.

ce sale fils de pvte d’Aulas https://t.co/kPZZwi739M

fils de monsieur crab’s

Comment ce compte de fils de pute a 5k et j'ai toujours pas 1k vous avez pas honte ?

Mon fils de pute de chat a chier dans ma chambre tfou 3hlik

Bande de fils de pute remboursez moi le déplacement https://t.co/cqPvliu5lt

THT est énorme fils de pute https://t.co/of5upL9LLP

Mdr l’autre fils de pute m’a supprimé du space dès qu’il est devenu cohôte

Je n'ai pas calicer de contrat satanique protocaves des sage des fils de putes a date , j'ai rien fais

Un vrai homme assiste à la naissance de son fils plutôt https://t.co/heqiGcJVyK

J’en peux plus bande de fils de putes

Le fils de l'homme

Genre en vrai c’est naz d’être un fils de pute parce que une fois qu’on sait que t’es un fils de pute t’es cramé partout

Gg, connecté direct, fil d'attente puis plus rien

Moi aussi j’étais direct file d’attente

Les théories sur les fans de cinéma se confirme : ce sont tous des fils de pute

Ferme ta geule fils de pute je troll ?

Toujours en file d'attente 🤡

Tu t ramasse avec ca en file a l épicerie 🛒 OMG 😳

FIFA JEU DE FILS DE PUTE

Mddddr vasy ftg fils de pute https://t.co/vdRMSM8uWr

Mon fils il fait ça ma femme elle s’assois derrière pour le retour https://t.co/if5Bdqpoor

Mais rt pas ce genre de trucs sale fils de pute

Sale fils de pute

L’enfant c’est meme pas sur c l’miens c’fils de pvte https://t.co/zd2iHxyJDM

Bande de fils de pute !!! Essayez pour voir 🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥

Fils de pute de lâche, j'ai grave la haine https://t.co/dHDB7W7OoH

Énorme fils de pute, faut lui baiser sa tante https://t.co/pLBKd2yevv

Voilà une file d'attente qui fait infiniment plus plaisir que les files des étudiants devant les épiceries solidaires.

Je comprend pas ce genre de gros fils de pute https://t.co/zV0ZfT4pdc

🇫🇷 Le fils de Dieu est devenu un homme pour faire des hommes des fils de Dieu 🙏@gedqu4tre #NoJesusNoLife

Ton petit fils travaillait chez la société Clauger?

Mon fils ne fera de sport il serait dans les études

Merite la prison ce fils de chien https://t.co/0eJFpYZBWV

le plus gros fils de chien de tout GOT

LA SIERRA LEONE BANDE DE FILS DE PUTES

J’ai trop envie de zouker avec le fils de quelqu’un sur "donne-le moi" 😩.

Quel fils de pute https://t.co/xtSZW2Y60o

Putain c'est qui le fils de pute qui a posé la question sur la RDC ?

Franchement les hommes sont des fils de pute j’ai pas les mots mdr

Mon fils fera du cheval plus tard

des gars veulent que leur fils devienne footballeur https://t.co/KaKXdQAsR4

devant le goss en plus c’est vraiment un fils de pute

23:30 et Madame fait la file au McDo 💀

Qui est le fils de pute qui a inventé Excel ?

On rappelle que tous ceux qui soutiennent bassem sont des fils de putes

cluster som 400 289:

Mon fils avec mark

Fils de pute https://t.co/GbThVyfWfZ https://t.co/2FjJfPq9fO

MAHITO SALE CHIEN LAISSE MON FILS

C'EST UN FILS DE CHIEN OMGGGGGGGGGG https://t.co/WCZ1p8UdqP

et ça va chui un fils de pute si j’le regarde pas

Les gars qui font ça sont aussi des fils de putes hein l'un n'empêche pas l'autre

Quel sale fils de pute mdr https://t.co/MldSAPhFFa

Un p'tit amis du fils sorros

Merci je vais filer la réf à mon caviste ; il a plein de bouteilles rigolotes dans son échoppe. https://t.co/4gqk4X1sRM

Fils de pute j'avais pas vu toi la #portos #rendnousleuro #smooth

Est ce que son fils recevait des parfums?

Quand Satan regarde le fils unique de Dieu https://t.co/CSXDsCknQw https://t.co/JGjTUn8dOm

Se fils de pute la https://t.co/DCjaY69zbN

Non non c’est un sale fils de pute https://t.co/UA8e3ib7Tp

Ton fils caché ça je reconnais

fils de pute 👇🏾 https://t.co/6Yj125tE3n

File chez le dentiste ! https://t.co/m9Sh9pppdW

Juste jean fils dsl mais j'ai jamais aimé

sale petit fils de puye je te baiss ta mère devant toi

seul peuple de fils de pute à faire ça https://t.co/l8w0qOzCUb

Même les fils et les filles des dirigeants du CAC 40 ? https://t.co/qlta4IyVwT

Quand un fils de pute essaie de défendre plein d autres fils de pute

Oui mais pas de fils de connard comme toi !

Comment un tel fils de pute peut être sur ce mur j’en reviens pas https://t.co/nJX2mQuMyQ

Génial pour mon fils il va adorer

Execution publique pour celui qui frappe et celui qui filme comme un sale fils de pute

ma femme, mon fils et moi au parc de la tête d'or https://t.co/ZAKC5HP5xQ

ah ouais? tu es le fils de Douk Saga aussi ?

Wshhh t’es le fils de douk saga toi ?c’est pour ça t’es un gahou comme ça

Je te pardonnes fils

Tu quoque mi fili

Allez mon fils sera ravi

Tu me dis ca avec ton fil twitter?

Avez vous un fils nommé Romain ?

C'est le fils à Maurice ? 😏

Le paf DE l'autre, CE FILS A PUTE

Colore ton fil https://t.co/tXgmKcLZey

BAHAHAHAHAH J'VIENS DE VOIR C QUOI CES ROUES DE FILS DE PUTE

BAJI MON FILS IL AIMAIT TROP SHIN https://t.co/5OrpKGhBBg

Va te faire enculer fils de pute

Cringe tu est un fils de merde

Son fils il est trop serein https://t.co/Ha2elnOmoz

des gros fils de pute

Les mecs qui jouent la montre en prolongations sur fut a la 100e vous êtes des fils de pute

Merci de remplir mon fil d'amour !

Fila jamais te critiquei

D'ailleurs Daniel si t'as les musiques de préhistoric Park en flac dans un coin je te file mon mail

Mais j’sui un fils de pute?????? https://t.co/FDxYRqhRni https://t.co/qQB8W9nFDF

Le fils d'Olier Mordrel, père du nationalisme breton. https://t.co/hHFCdSPjOO

Millibyte files https://t.co/bHYl0bgUDs

T’es hideux fils de pute Reste trkl sur internet ptit pecno

NON CASSE TOI LAISSE LE LIRE ESPÈCE DE FILS DE PUTE

tout les jours fuck le chevaucheur de cochon truc que des fils de pute

Le fils spirituel de Sarkozy 😂

Il a un fil à la place des lèvres mdrr

on va se battre fils

mon fils il fera du foot.

File moi ton boule oui 🥰

CELUI QUI TOUCHE MON FILS J'LE DEFONCE LA RASS OK ?

Pas de quoi fils 🤣

Remplaçons « fils de pute » par « fils de chien » les filles. Arrêtons avec les mères.

La fille de sanna et mon fils ça je reconnais https://t.co/miXLYMHmRp

tel père tel fils ☝️ https://t.co/IHs0SaEEBd https://t.co/1LhzIR17UM

Les Marroquìn ce sont des détraqués. Le père, le fils et le petit-fils ce sont tous des malades 🤮

Sale fils de pute https://t.co/9U1v3velsl

ce sale fils de pvte d’Aulas https://t.co/kPZZwi739M

Merci c'est le fils de mon cheikh

fils de monsieur crab’s

Quand la var vous donner des pénalty t'ouvrer pas tageul fils de pute

Comment ce compte de fils de pute a 5k et j'ai toujours pas 1k vous avez pas honte ?

File a exemption

'Quatre roues motrices, fils'

Mon fils

Sur le fil du rasoir

Mon fils de pute de chat a chier dans ma chambre tfou 3hlik

"tu es mon fils Naruto"Woaaaaaaaa

FILS DE PUTE DEMAIN JE LE FARM

Un fils de pute mais une belle pute de ça race ûn fils de putr… une salopette inca à le

Je suis toujours le même fils de pute du début, rien n'a changé

Bande de fils de pute remboursez moi le déplacement https://t.co/cqPvliu5lt

J'étais juste un gentil fils 😂 en tout cas là dessus

fils tu déconnes ?

G envie de vomir de fils de pute

C’est vraiment une équipe de fils de pute.

THT est énorme fils de pute https://t.co/of5upL9LLP

Pas trop de confiance dans ce fils de pute. https://t.co/3N44nde8W4

C’est normal c’est un fils de pute tu crois quoi ? https://t.co/Fyn6rENtW1

Fils de pute de Thibs https://t.co/7ah26mygmj

Quasiment plus depuis avant la naissance de son second fils😭🤣

EH le fils a 😭 https://t.co/UVoxvCbpOV

Mdr l’autre fils de pute m’a supprimé du space dès qu’il est devenu cohôte

Ta grand mère la fils de pute https://t.co/u2hKwDuUbp

Je n'ai pas calicer de contrat satanique protocaves des sage des fils de putes a date , j'ai rien fais

ce fil🤌🏽 https://t.co/rqDn3s6sG1

filé mignon 🗣️😂

TU ES NUL FILS DE PUTE

[EN DIRECT] Le Fil 22 avec c'est maintenant: https://t.co/yDcJCrjlWD https://t.co/dj6ppk38uM

Edit file scan pdf https://t.co/N9BqxvF3jC

Filé

HK concert au fil de l'eau https://t.co/pSsCHY6eyh via

Mon fils ma bataille Daniel Balavoine

andré michel Fils de chien

Portland c'est très nul, si on la laisse filer c'est abusé

me on fil essays: https://t.co/YTgq833hNj

Mon fils et nude la

Mais comment tu peux être des fils de pute pareil https://t.co/gAkdADjBpZ

Pourquoi j'ai l'impression que ce fils de pute c'est vraiment Danuel House ce tdc https://t.co/ngt41bhBo0

Ayton sale fils de pute va

Un vrai homme assiste à la naissance de son fils plutôt https://t.co/heqiGcJVyK

BONJOUR JE SUIS UN SOMBRE FILS DE PUTE

J’en peux plus bande de fils de putes

Liberté pour les ultras ligue des fils de pute 🖕🖕

Le fils de l'homme

Notre fils h24 : https://t.co/gUbDIAuBhv

Votre fils est contient de la réalité !

éduquez vos fils https://t.co/gEfIJsCHLc

La file d’attente est en pause

Genre en vrai c’est naz d’être un fils de pute parce que une fois qu’on sait que t’es un fils de pute t’es cramé partout

Gg, connecté direct, fil d'attente puis plus rien

Pour celles et ceux qui attendent encore dans la file : https://t.co/tlONinExOk

Moi aussi j’étais direct file d’attente

Excellent, mon fils 🤭 https://t.co/A3q7dyjfBh

L'obscurantisme comme fils des Lumieres

Putain, mon fils va me réclamer un smartphone à mille balles, maintenant. "Moi aussi j'en veux un, pourquoi j'en ai pas moi ?".

Il y aurait 752821 personnes dans la file d’attente 😭 https://t.co/MA52gVxEZZ

Fils tu te calme.

les fils de putes dans le bus on espère une mort imminente

Les théories sur les fans de cinéma se confirme : ce sont tous des fils de pute

Écrivez moi si possible pour que je vous file le numéro s’il vous plaît. Merci.

Jacquemain est le fils de Marie-Antoinette.

C'est aussi le problème du Congo actuellement. Vous n'avez pas à vous plaindre. Ce que voys vivez comme fils du Kasaï, nous le vivons comme fils de la RDC.#MbokaYaBanaNaBana

💡 Benoît Hamon n'est pas le fils de Toutank.

On dirait le fils de yassencore et Jlbalty

La file d’attente pour L’ALLEMAGNE a débuté https://t.co/O7wa05i6y3

D&amp;D si maman dormait même si son fils rentrait pas.

Je suis dans la file depuis 9h 🤞🏻 Je stresse tellement

C’était donc toi Ferland, assassin, fils de pute https://t.co/rhv5z4t3na

Waow… Elle ne sera pas glaciale avec mon fil Twitter dans la poche… 🥵🥵

Tu files un mauvais coton toi

Wahh le clash 😂😂😂😂 Tu as quel âge fils de catin ? Vas-y ramène ta mère qu'on te refasse

Ça serait top que tu fasses un fil plus complet

Imagine ma main dans ta gueule fils de chien.

Ferme ta geule fils de pute je troll ?

Mais filez leurs une cornemuse !

File d'attente 

C’est une blague !! J’étais dans la file depuis 9h et y’a plus rien !!

Toujours en file d'attente 🤡

Lewandowski était injouable ce fils de pute

🇫🇷 Marlène , vous laisser filer l'IMC .🇫🇷@DanielPilotte1 https://t.co/yWgxWlMy3H

😀😀 un fils de pute

J’ai passé la file d’attente mais j’ai ça .. https://t.co/VU4Kji7J7P

Choisi bien fils... https://t.co/AHhCBzpCF8

ok espece de fils de ta mere la gentil qui a un fils extraordinaire que je kiff trop

Je file regarder des tiddi…tutos 😇

🍌🍌🍌quand tu n’étais pas devenu son père à elle, elle était le 2 à la fois ! Moi je reste son fils à vie https://t.co/e0ClVfXn18

Ça c’est mon fils quand il voyage il est trop beau 🥺 https://t.co/XyJL5wpJHJ

punaise la file d’attente pour birmingham

Ferme ta gueule fils de pute

Père, fils et Saint-Esprit c'est le seigneur Jésus-Christ.

Oui fils

Fils indigne ✊🤣 https://t.co/Xk43jnTtwk

C'est surtout le fils de miyazaki qui essaye des nouveautés

j’veux que mon fils soit comme lui https://t.co/BHtTaDTMvn

Fils de pute , c’est ça ?

Un fils de pute de démon https://t.co/nRXlOgdA74

Mon fils m'a dit: "des raviolis avec un truc gris dedans" 🧐

Sale fils de

C pour etre un fils de p.te ca https://t.co/nxqGsELbn8

Quel fils de pute ...

Faire la file pour un sandwich 😓

mrci toi aussi mais c toi mn fils dcp

Des air pods , ou d'autres ecouteurs sans fil

MDRRRRRR ON TE PARLE FILS DE PUTE https://t.co/xhSiz6JfYV

Ton fils qui traîne à rg

quelle fils de pute 😡😡🤬🤬

Tu t ramasse avec ca en file a l épicerie 🛒 OMG 😳

Les étoiles filantes de

oklm y'a un plus gros fils de pute derrière moi https://t.co/Db2w6x2V7y

trop frais mes fils

Faut le crever ce fils de pute

ces fils de pute n'ont honte de rien !

PPPTDRRRKRKRKRKR oui je suis le fils à elodie costa

W'Allah il est mm pas drôle en plus ce fils de pute https://t.co/zFU7QSPsOZ

Voir ce fil : https://t.co/fjftLlQKfI

Fils de pute https://t.co/SMvYPWUnxm

Mais quel fils de pute… https://t.co/pWKcY3z6mk

FIFA JEU DE FILS DE PUTE

svp bannissez moi de votre jeu de fils de pute de merde merci

Mais foutez moi ce fils de pute en taule https://t.co/4mdZOrev7j

file noise complaint

Vraiment pas le fils de son père

Sale fils de pute on espère une peine lourde https://t.co/dsOesBSfAb

Y’a quoi de raciste fils de pute??

Ptn 😭😭😭 Mtn je suis en file d'attente, la poisse !!

Le fils de Drucker https://t.co/Tdnk0GCBYZ

Fils de pute

Mddddr vasy ftg fils de pute https://t.co/vdRMSM8uWr

A mon fils pour ma mère mais là monnaie est hram

Le fils de pute https://t.co/7j4N1u3vWs

Eh je suis ko, tu tape un délire sur mon fils déjà

GG fils

Essaye d’inventer un mot fils de pute

Le fils de pute

J'ai un peu perdu le fil... Dîtes-moi , on en est à combien d'utilisateurs de l'application ?

Un bon truc de fils de lache https://t.co/Od919IuVx5

Fils de pute. https://t.co/XTvkLivZlK

Mon fils est beau #JeudiConfession

Ma chambre un chantier du fil partout

Ptddrr je croyais que j’étais la seul, quand c’est sur mon fils je me vexe vite 😭

La peine de mort pour ces fils de pute c'est même pas à la hauteur de ce qu'ils méritent. https://t.co/zeM8GZljit

Mon fil Twitter....🤣 https://t.co/q3rMnDUbqR

Jmen fou fils

arnaque fils de pute https://t.co/05nJu3X0Ov

Meme la prison c’est trop peu pour ces fils de putes, qu’on leur coupe les couilles merde https://t.co/yn4pfKnv9T

Il ressemble a mon fils laisser le entrer https://t.co/isLSBvQdJD

Vu que c'est trop compliqué de suivre le fil d'un sujet en 2 clics, tiens : https://t.co/Q3Ifo4wjh0

Ds ma famille on a des fils en premier

Pourquoi les files quand elle sont amoureuses elle sont bêtes comme ça? https://t.co/QlKFmlsnF7

Mon fils il fait ça ma femme elle s’assois derrière pour le retour https://t.co/if5Bdqpoor

Mais rt pas ce genre de trucs sale fils de pute

Fils de pute ! https://t.co/unoDybEqIT

Au fil de l'eau https://t.co/tTOMTlVwte

Devant le gosse en plus quel fils de timpe https://t.co/2pR00iWDKc

Sale fils de pute

Le fil de l'angoisse pour moi https://t.co/aC36DHiPvB

c'était hier. J'attends qu'elle s'en remette, car à priori son fils aussi était aux urgences. L'un après l'autre, paf.

Regardez moi ce fils de chien battu… tsuipss https://t.co/I6lAuULyAK

fils de pute https://t.co/b1nGtWlZmf

Quelqu'un pr lui filer sa dose?

Fils de pute https://t.co/ESfiHV9fPU

Il a un fils en zonzon ?

tel père tel fils https://t.co/NDQo8sAM6g

Gros fils de pute du matin au soir aussi

L’enfant c’est meme pas sur c l’miens c’fils de pvte https://t.co/zd2iHxyJDM

Quel fils de pvte c fou la haine https://t.co/3z3gLUVFc9

Bande de fils de pute !!! Essayez pour voir 🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥🔥

Fils de pute de lâche, j'ai grave la haine https://t.co/dHDB7W7OoH

Peine de mort fils de pute https://t.co/89s7N6RTM4

File dans le garage et puis démarre la 307

Ce gros fils de pute la https://t.co/hlKt9iGNaa

Je lui briserai les couilles à de fils de pute mais non il en a pas ce gros enculé

Ok le fils de pute

On est sur un high level de fils de puterie la https://t.co/kiJ255PDZa

qu’on lui fasse bouffer ses couilles ce fils de pute https://t.co/00eVtWn3CD

Peu importe le contexte c’est un immense fils de pute mdr on s’en fout

C’est quel genre de fils de pute?

Énorme fils de pute, faut lui baiser sa tante https://t.co/pLBKd2yevv

C’est un fils de pute je l’aime pas

ces fils de putes ils me mettent l’œil

un fils de pute

Voilà une file d'attente qui fait infiniment plus plaisir que les files des étudiants devant les épiceries solidaires.

T’as oublié de lui filer sa sucrette pour le récompenser 🤡

Quelle fils de pute

La vie des funambules elle tient à un fil

Vous êtes des voleurs de fils de pute

Chronos quand il mangeait ses fils à la naissance https://t.co/o4XYmKsaqu https://t.co/iIqdN3prxH

Fils de pute de chinois https://t.co/we70ad2ba3

Sur le fil la les beaufs se lâchent #Bobia https://t.co/bRCraJtp9O

Peine de mort pour ce fils de pute https://t.co/RL4PLd8MTq

😂😂😂😂 ce fils deup

Fils de pute ,

Mais quel sale fils de pute https://t.co/8uZaYLRuGW

Gros fils de pute https://t.co/3E6cdtElVU

Je résilie l’abonnement fils de pute

je suis très fière fils

YA QUE DES FILS DA PVTE DANS LE SECTEUR

Je comprend pas ce genre de gros fils de pute https://t.co/zV0ZfT4pdc

Les digne fils du pays

Ma mère a retrouvé son petit fils je n’existe plus

Fils de pute https://t.co/KZoC5H1Dhh

mon fils https://t.co/ve6a8H62N8

gros chien, j’espère quelqu’un le retrouve ce fils de pute https://t.co/SHpoJNfavu

Ferland Mendy petit fils de pute

🇫🇷 Le fils de Dieu est devenu un homme pour faire des hommes des fils de Dieu 🙏@gedqu4tre #NoJesusNoLife

FILS DE PUTE 😮😮😮😮😮😮😮

la fifa bande de fils de pute

Le fils de pute. https://t.co/4waiY6oTir

C’est sur et certain fils mdrrr

Protégeons nous frères, neveux, fils…

C’est trop grave c’est fils de pute https://t.co/3D3ZtGBba6

Mon fils ne sera pas footballeur professionnel https://t.co/BcDDy24NQK

Abdoulaye quand il s’agit d’être un fils de pute https://t.co/yiHDO87bqF https://t.co/EohmM4APvT

Fils de pute https://t.co/y6Z6YMLCmD

Bande de fils se pute va

La fédération française vous êtes des fils de pute

Qu'il aille dire ça au mari et au fils de #Mauricette ce tordu !

Ton petit fils travaillait chez la société Clauger?

Mon fils ne fera de sport il serait dans les études

Merite la prison ce fils de chien https://t.co/0eJFpYZBWV

le plus gros fils de chien de tout GOT

C’est que des fils à papa https://t.co/8PwqCOAjvM

courage à votre fils

LA SIERRA LEONE BANDE DE FILS DE PUTES

Fils de pute ta pas d’info sur le Barça

fils de pute va https://t.co/wWTQtc8KVa

Mdrrrr mon fils fera du basket

ta pas honte fils de pute

quel fils de pute https://t.co/5f2mb4Sp6O

t’es vraiment un sacré fils de pute

Bh si demain y’a l’école fils

De toute façon dans chaque branche du sport ou de n’importe quel domaine ya des fils de putes de pédophiles et de violeurs

Au parloir visiter ses deux fils

La vrai question est de savoir quel fils de pute de fouteux est derrière ce compte ? https://t.co/ECuLhdNlGj

daccord tu me files le lien si tu trouves

Et voilà le fils de pute on l’attendait

J’ai trop envie de zouker avec le fils de quelqu’un sur "donne-le moi" 😩.

Macron fils de pute

ça c ma réaction quand je te vois sur mon file d’actualité 🥺✨

Quoi Molina ? antoi*ne griezm*ann est un fils de pute ?!🙀

Le fils de Sassou ? Mdr #SpaceMolina

Il va que très peu sur twitter Mon fils

Quel fils de pute https://t.co/xtSZW2Y60o

Quand on dit le fils de… il a pas qu’un fils, on parle de qui ? 🤣 le seul et l’unique ?

Putain c'est qui le fils de pute qui a posé la question sur la RDC ?

Franchement les hommes sont des fils de pute j’ai pas les mots mdr

ftg toi pourquoi tu me mentionne pour dire des truc de fils de putes

Des grands fils de pute

mon fils sera du basket ou du golf hein

filé 🙏🏾🙏🏾

fils de pute https://t.co/sqNhvU3Z6L

Quel immonde fils de pute sérieux

Ignobles fils de pute

Mon fils fera du cheval plus tard

ROOOH c’est qui ce fils de pute

Ptdrrrr quel fils de timp

J’ai honte des noirs que donne la parole à des fils de pute comme Bassem

un gros fils de pute celui là 🤦🏽‍♂️

Tema ce fils de timp

des gars veulent que leur fils devienne footballeur https://t.co/KaKXdQAsR4

Mon fils veut pas dodo 😩😩

Le Fils Du Masque, a masterpiece https://t.co/zNLRaoF6yx

devant le goss en plus c’est vraiment un fils de pute

Eh j'suis fan de mon fils il me fait trop rire 😭😭💚💚💚💚 https://t.co/sDZLWNlj4p

Fils de p*te https://t.co/ySLXenEWAS

fut quel jeu de fils de pute

23:30 et Madame fait la file au McDo 💀

Digne fils du lualaba 🙏J ai confiance 👌

Oh putain ! File moi une maladie anale Dimitri ❤️

Il ne seraient pas plutôt de plus en plus nombreux à vous prendre pour un fil d’intox nauséabond ?

Les fils de Soros ? https://t.co/wsRjMnMt3H

sa me dégoûte jvoit des gens c’est des fils de pute

Mon fil d'actualité est facétieux ce matin. https://t.co/J67FrR3r5Q

Peine de mort pour ce fils de pute https://t.co/iZ2Jr3ultc

Hernandez ? Bah fils de pute et c’est pas mon joueurs préféré mdrrr

Tu as combien, je te file ça chap même 🤭

Comment tu peux être un énorme fils de pute comme ça j’ai la haine https://t.co/yW7XdM05nS

Y a une file d’attente pour aujourd’hui ? Sur ticketmaster ?

mais du coup scaramouche c plus son fils ou son frère ???? https://t.co/eKuBovFtgt

la file d'attente a commencé

Fallait qu'il se ramène ce fils de pxte de compétition. https://t.co/o66kTXBZdS

Y est de retour celui-là ? Pour la naissance de son fils sûrement 🤣🤣🤣

Lectures sur le fil https://t.co/ZJPupygRfV https://t.co/Wkz1WSn8hd

Qui est le fils de pute qui a inventé Excel ?

La file d’attente commence a partir de quand? 😭

On rappelle que tous ceux qui soutiennent bassem sont des fils de putes

Mes fils sont heureux alors je le suis aussi 🤱🏾🤎 https://t.co/EldtGvpfuh

---

_Remarque:
 Une nouvelle fois les résultats sont décevant, mais si on observe le cluster 289 de som 400 on constate que les outliers locaux contenant "fils de" sont bien sélectionnés parmi un cluster de fil, fille, filles, fils etc._ 
 
---

1M_Min_clusters:

    km25:
        lof kNeighbors = 2:
        cluster 4                  57               1.6011%             Expressions prépondérantes: pétasse
        lof kNeighbors = 3:
        cluster 4                  46               2.2137%             Expressions prépondérantes: pétasse
        lof kNeighbors = 4:
        cluster 4                  49               3.1654%             Expressions prépondérantes: pétasse
        
    km50:
        lof kNeighbors = 2:
        lof kNeighbors = 3:
        lof kNeighbors = 4:
        
    km100:
        lof kNeighbors = 2:
        cluster 16                   8              1.5873%             Expressions prépondérantes: *
        cluster 30                   2              1.0363%             Expressions prépondérantes: *
        cluster 35                  41              4.9818%             Expressions prépondérantes: pétasse, pute
        cluster 64                  28              1.1859%             Expressions prépondérantes: pute
        cluster 86                   5              1.6026%             Expressions prépondérantes: *
        cluster 90                   3              1.3761%             Expressions prépondérantes: *
        lof kNeighbors = 3
        cluster 16                   8              2.7397%             Expressions prépondérantes: *
        cluster 30                   2              1.0753%             Expressions prépondérantes: *
        cluster 35                  42              6.4715%             Expressions prépondérantes: pétasse
        cluster 64                  16              1.6771%             Expressions prépondérantes: pute
        cluster 86                   4              1.7316%             Expressions prépondérantes: * 
        cluster 90                   3              1.8634%             Expressions prépondérantes: *
        lof kNeighbors = 4:
        cluster 16                   9              4.0541%             Expressions prépondérantes: *
        cluster 35                  44              7.6389%             Expressions prépondérantes: pétasse
        cluster 64                   5              1.1682%             Expressions prépondérantes: *
        cluster 86                   4              2.5157%             Expressions prépondérantes: *  
        cluster 90                   3              2.3077%             Expressions prépondérantes: *   
        
    som100:
        lof kNeighbors = 2:
        cluster  6                   4              1.1869%             Expressions prépondérantes: * 
        cluster 26                   6              1.1321%             Expressions prépondérantes: *  
        cluster 34                  49             14.9847%             Expressions prépondérantes: pétasse, pute
        cluster 63                  45              1.3353%             Expressions prépondérantes: pute
        cluster 66                   5              1.462%              Expressions prépondérantes: * 
        cluster 76                  12              2.6906%             Expressions prépondérantes: *
        lof kNeighbors = 3:
        cluster  6                   3              1.4925%             Expressions prépondérantes: * 
        cluster 26                   4              1.1364%             Expressions prépondérantes: * 
        cluster 34                  49             19.8381%             Expressions prépondérantes: pétasse
        cluster 63                  13              1.0726%             Expressions prépondérantes: pute
        cluster 66                   6              2.6549%             Expressions prépondérantes: *
        cluster 76                  13              4.2763%             Expressions prépondérantes: *
        lof kNeighbors = 4:
        cluster  6                   3              2.0548%             Expressions prépondérantes: *
        cluster 26                   3              1.1278%             Expressions prépondérantes: *
        cluster 34                  49             21.4912%             Expressions prépondérantes: pétasse
        cluster 66                   8              4.9383%             Expressions prépondérantes: *
        cluster 76                   9              4.5226%             Expressions prépondérantes: * 
        
    som225:
        lof kNeighbors = 2:
        cluster 30                  27              9.4077%             Expressions prépondérantes: pute
        cluster 38                   1              1.1111%             Expressions prépondérantes: *
        cluster 77                  36              8.867%              Expressions prépondérantes: pétasse
        cluster 82                  15              1.385%              Expressions prépondérantes: pute
        cluster 143                  4              1.3841%             Expressions prépondérantes: *
        cluster 151                  5              3.3333%             Expressions prépondérantes: *
        lof kNeighbors = 3:
        cluster 30                  31             15.2709%             Expressions prépondérantes: pute
        cluster 77                  36             10.2273%             Expressions prépondérantes: pétasse
        cluster 82                   7              1.5766%             Expressions prépondérantes: *             
        cluster 151                  1              1.1494%             Expressions prépondérantes: *
        lof kNeighbors = 4:
        cluster 30                  25             15.1515%             Expressions prépondérantes: pute
        cluster 77                  36             11.0769%             Expressions prépondérantes: pétasse
        cluster 82                   5              2.4876%             Expressions prépondérantes: *
        cluster 151                  3              3.2967%             Expressions prépondérantes: *
        
    som400:
        lof kNeighbors = 2: 
        cluster 7                    2              2.1739%             Expressions prépondérantes: *
        cluster 58                  11              1.3237%             Expressions prépondérantes: *
        cluster 72                  11              1.7799%             Expressions prépondérantes: *
        cluster 78                   5              1.199%              Expressions prépondérantes: *
        cluster 107                  1              1.1364%             Expressions prépondérantes: *
        cluster 119                  3              1.145%              Expressions prépondérantes: *
        cluster 124                  3              1.6949%             Expressions prépondérantes: *
        cluster 153                 20             12.8205%             Expressions prépondérantes: pute    
        cluster 161                 35             10.8025%             Expressions prépondérantes: pétasse
        cluster 176                  2              2.9851%             Expressions prépondérantes: * 
        lof kNeighbors = 3:
        cluster 7                    1              1.6667%             Expressions prépondérantes: *
        cluster 58                   4              1.2821%             Expressions prépondérantes: *
        cluster 72                   4              1.4493%             Expressions prépondérantes: *
        cluster 78                   1              0.4762%             Expressions prépondérantes: *                
        cluster 124                  3              2.2727%             Expressions prépondérantes: *
        cluster 153                 10              8.547%              Expressions prépondérantes: pute 
        cluster 161                 35             12.9151%             Expressions prépondérantes: pétasse   
        lof kNeighbors = 4: 
        cluster 7                    1              1.9608%             Expressions prépondérantes: *
        cluster 58                   3              2.1429%             Expressions prépondérantes: *
        cluster 124                  1              1.0753%             Expressions prépondérantes: *
        cluster 153                 17             13.6%                Expressions prépondérantes: pute 
        cluster 161                 35             14.5833%             Expressions prépondérantes: pétasse              

---